# Why feature learning is better than simple propositionalization

_NOTE: Due to featuretools's and tsfresh's memory requirements, this notebook will not run on binder when RUN_FEATURETOOLS=True RUN_TSFRESH=True._

In this notebook we will compare getML to featuretools and tsfresh, both of which open-source libraries for feature engineering. We find that advanced algorithms featured in getML yield significantly better predictions on this dataset. We then discuss why that is.

Summary:

- Prediction type: __Regression model__
- Domain: __Air pollution__
- Prediction target: __pm 2.5 concentration__
- Source data: __Multivariate time series__
- Population size: __41757__

*Author: Dr. Patrick Urbanke*

## Background

Many data scientists and AutoML tools use propositionalization methods for feature engineering. These propositionalization methods usually work as follows:

- Generate a large number of hard-coded features
- Use feature selection to pick a percentage of these features

By contrast, getML (https://getml.com/product) contains approaches for feature learning: Feature learning adapts machine learning approaches such as decision trees or gradient boosting to the problem of extracting features from relational data and time series.

In this notebook, we will benchmark getML (https://getml.com/product) against featuretools (https://www.featuretools.com/) and tsfresh (https://tsfresh.readthedocs.io/en/latest/). Both of these libaries use propositionalization approaches for feature engineering.

As our example dataset, we use a publicly available dataset on air pollution in Beijing, China (https://archive.ics.uci.edu/ml/datasets/Beijing+PM2.5+Data). The data set has been originally used in the following study:

Liang, X., Zou, T., Guo, B., Li, S., Zhang, H., Zhang, S., Huang, H. and Chen, S. X. (2015). Assessing Beijing's PM2.5 pollution: severity, weather impact, APEC and winter heating. Proceedings of the Royal Society A, 471, 20150257.

We find that getML significantly outperforms featuretools and tsfresh in terms of predictive accuracy (**R-squared of 62.3%** vs **R-squared of 50.4%**).

Our findings indicate that getML's feature learning algorithms are better at adapting to data sets and are also more scalable due to their lower memory requirement.

## A web frontend for getML

The getML monitor is a frontend built to support your work with getML. The getML monitor displays information such as the imported data frames, trained pipelines and allows easy data and feature exploration. You can launch the getML monitor [here](http://localhost:1709).

In [1]:
import os
from urllib import request

import getml
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import pearsonr

from utils.load import load_or_retrieve

%matplotlib inline

In [2]:
RUN_FEATURETOOLS = False
RUN_TSFRESH = False

if RUN_FEATURETOOLS:
    from utils import FTTimeSeriesBuilder

if RUN_TSFRESH:
    from utils import TSFreshBuilder

## 1. Loading data

### 1.1 Download from source

We begin by downloading the data from the UCI Machine Learning repository.

### 1.2 Prepare data for tsfresh and getML

Our our goal is to predict the pm2.5 concentration from factors such as weather or time of day. However, there are some **missing entries** for pm2.5, so we get rid of them.

We then load the data into the getML engine. We begin by setting a project.

In [3]:
getml.set_project("air_pollution")



Connected to project 'air_pollution'


In [4]:
data = getml.datasets.load_air_pollution()


Loading population...


We also want to split our data into a training and testing set.

In [5]:
split = getml.data.split.time(
    population=data, time_stamp="date", test=getml.data.time.datetime(2014, 1, 1)
)

split

,
0,train
1,train
2,train
3,train
4,train
,...


## 3. Predictive modeling


### 3.1 Pipeline 1: Complex features, 7 days

For our first experiment, we will learn complex features and allow a memory of up to seven days. That means at every given point in time, the algorithm is allowed to back seven days into the past.

getML uses relational learning to build construct the pipelines. Even though there is a simpler time series API, the relational API is more flexible which is why decide to use it.

In [6]:
time_series1 = getml.data.TimeSeries(
    population=data,
    alias="population",
    split=split,
    time_stamps="date",
    memory=getml.data.time.days(7),
)

time_series1

data model

  population:
    columns:
    - DEWP: numerical
    - TEMP: numerical
    - PRES: numerical
    - Iws: numerical
    - Is: numerical
    - ...

    joins:
    - right: 'population'
      time_stamps: (population.date, population.date)
      relationship: 'many-to-many'
      memory: 604800.0
      lagged_targets: False

  population:
    columns:
    - DEWP: numerical
    - TEMP: numerical
    - PRES: numerical
    - Iws: numerical
    - Is: numerical
    - ...


container

  population
      subset   name          rows   type
  0   test     population    8661   View
  1   train    population   33096   View

  peripheral
      name          rows   type     
  0   population   41757   DataFrame

In [7]:
relmt = getml.feature_learning.RelMT(
    num_features=10,
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
    seed=4367,
    num_threads=1,
)

predictor = getml.predictors.XGBoostRegressor(n_jobs=1)

pipe1 = getml.pipeline.Pipeline(
    tags=["memory: 7d", "complex features"],
    data_model=time_series1.data_model,
    feature_learners=[relmt],
    predictors=[predictor],
)

pipe1

Pipeline(data_model='population',
         feature_learners=['RelMT'],
         feature_selectors=[],
         include_categorical=False,
         loss_function=None,
         peripheral=['population'],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['memory: 7d', 'complex features'])

It is good practice to always check your data model first, even though `check(...)` is also called by `fit(...)`. That enables us to make last-minute changes.

In [8]:
pipe1.check(time_series1.train)

Checking data model...

Staging...
[========================================] 100%


OK.


We now fit our data on the training set and evaluate our findings, both in-sample and out-of-sample.

In [9]:
pipe1.fit(time_series1.train)

Checking data model...

Staging...
[========================================] 100%


OK.

Staging...
[========================================] 100%

RelMT: Training features...
[========================================] 100%

RelMT: Building features...
[========================================] 100%

XGBoost: Training as predictor...
[========================================] 100%


Trained pipeline.
Time taken: 0h:6m:10.590749



Pipeline(data_model='population',
         feature_learners=['RelMT'],
         feature_selectors=[],
         include_categorical=False,
         loss_function=None,
         peripheral=['population'],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['memory: 7d', 'complex features', 'container-GiWZEb'])

url: http://localhost:1709/#/getpipeline/air_pollution/sinbqF/0/

In [10]:
pipe1.score(time_series1.test)


Staging...
[========================================] 100%

RelMT: Building features...
[========================================] 100%




,date time,set used,target,mae,rmse,rsquared
0,2021-08-23 08:05:54,train,pm2.5,35.1664,50.9038,0.6925
1,2021-08-23 08:06:23,test,pm2.5,39.6596,57.5014,0.6306


### 3.2 Pipeline 2: Complex features, 1 day

In [11]:
time_series2 = getml.data.TimeSeries(
    population=data,
    alias="population",
    split=split,
    time_stamps="date",
    memory=getml.data.time.days(1),
)

time_series2

data model

  population:
    columns:
    - DEWP: numerical
    - TEMP: numerical
    - PRES: numerical
    - Iws: numerical
    - Is: numerical
    - ...

    joins:
    - right: 'population'
      time_stamps: (population.date, population.date)
      relationship: 'many-to-many'
      memory: 86400.0
      lagged_targets: False

  population:
    columns:
    - DEWP: numerical
    - TEMP: numerical
    - PRES: numerical
    - Iws: numerical
    - Is: numerical
    - ...


container

  population
      subset   name          rows   type
  0   test     population    8661   View
  1   train    population   33096   View

  peripheral
      name          rows   type     
  0   population   41757   DataFrame

In [12]:
relmt = getml.feature_learning.RelMT(
    num_features=10,
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
    seed=4367,
    num_threads=1,
)

predictor = getml.predictors.XGBoostRegressor(n_jobs=1)

pipe2 = getml.pipeline.Pipeline(
    tags=["memory: 1d", "complex features"],
    data_model=time_series2.data_model,
    feature_learners=[relmt],
    predictors=[predictor],
)

pipe2

Pipeline(data_model='population',
         feature_learners=['RelMT'],
         feature_selectors=[],
         include_categorical=False,
         loss_function=None,
         peripheral=['population'],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['memory: 1d', 'complex features'])

In [13]:
pipe2.check(time_series2.train)

Checking data model...

Staging...
[========================================] 100%


OK.


In [14]:
pipe2.fit(time_series2.train)

Checking data model...

Staging...
[========================================] 100%


OK.

Staging...
[========================================] 100%

RelMT: Training features...
[========================================] 100%

RelMT: Building features...
[========================================] 100%

XGBoost: Training as predictor...
[========================================] 100%


Trained pipeline.
Time taken: 0h:2m:5.558856



Pipeline(data_model='population',
         feature_learners=['RelMT'],
         feature_selectors=[],
         include_categorical=False,
         loss_function=None,
         peripheral=['population'],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['memory: 1d', 'complex features', 'container-6OCiAz'])

url: http://localhost:1709/#/getpipeline/air_pollution/i9l9JF/0/

In [15]:
pipe2.score(time_series2.test)


Staging...
[========================================] 100%

RelMT: Building features...
[========================================] 100%




,date time,set used,target,mae,rmse,rsquared
0,2021-08-23 08:08:34,train,pm2.5,38.1593,55.3541,0.6366
1,2021-08-23 08:08:44,test,pm2.5,47.5451,66.9418,0.4901


### 3.3 Pipeline 3: Simple features, 7 days

For our third experiment, we will learn simple features and allow a memory of up to seven days.

In [16]:
time_series3 = getml.data.TimeSeries(
    population=data,
    alias="population",
    split=split,
    time_stamps="date",
    memory=getml.data.time.days(7),
)

time_series3

data model

  population:
    columns:
    - DEWP: numerical
    - TEMP: numerical
    - PRES: numerical
    - Iws: numerical
    - Is: numerical
    - ...

    joins:
    - right: 'population'
      time_stamps: (population.date, population.date)
      relationship: 'many-to-many'
      memory: 604800.0
      lagged_targets: False

  population:
    columns:
    - DEWP: numerical
    - TEMP: numerical
    - PRES: numerical
    - Iws: numerical
    - Is: numerical
    - ...


container

  population
      subset   name          rows   type
  0   test     population    8661   View
  1   train    population   33096   View

  peripheral
      name          rows   type     
  0   population   41757   DataFrame

In [17]:
fast_prop = getml.feature_learning.FastProp(
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
    num_threads=1,
    aggregation=getml.feature_learning.FastProp.agg_sets.All,
)

predictor = getml.predictors.XGBoostRegressor(n_jobs=1)

pipe3 = getml.pipeline.Pipeline(
    tags=["memory: 7d", "simple features"],
    data_model=time_series3.data_model,
    feature_learners=[fast_prop],
    predictors=[predictor],
)

pipe3

Pipeline(data_model='population',
         feature_learners=['FastProp'],
         feature_selectors=[],
         include_categorical=False,
         loss_function=None,
         peripheral=['population'],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['memory: 7d', 'simple features'])

In [18]:
pipe3.check(time_series3.train)

Checking data model...

Staging...
[========================================] 100%


OK.


In [19]:
pipe3.fit(time_series3.train)

Checking data model...

Staging...
[========================================] 100%


OK.

Staging...
[========================================] 100%

FastProp: Trying 330 features...
[========================================] 100%

FastProp: Building features...
[========================================] 100%

XGBoost: Training as predictor...
[========================================] 100%


Trained pipeline.
Time taken: 0h:2m:8.736899



Pipeline(data_model='population',
         feature_learners=['FastProp'],
         feature_selectors=[],
         include_categorical=False,
         loss_function=None,
         peripheral=['population'],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['memory: 7d', 'simple features', 'container-dxhksP'])

url: http://localhost:1709/#/getpipeline/air_pollution/7hfWou/0/

In [20]:
pipe3.score(time_series3.test)


Staging...
[========================================] 100%

FastProp: Building features...
[========================================] 100%




,date time,set used,target,mae,rmse,rsquared
0,2021-08-23 08:10:58,train,pm2.5,36.2407,50.9318,0.7019
1,2021-08-23 08:11:08,test,pm2.5,46.3686,63.8692,0.543


### 3.4 Pipeline 4: Simple features, 1 day

For our fourth experiment, we will learn simple features and allow a memory of up to one day.

In [21]:
time_series4 = getml.data.TimeSeries(
    population=data,
    alias="population",
    split=split,
    time_stamps="date",
    memory=getml.data.time.days(1),
)

time_series4

data model

  population:
    columns:
    - DEWP: numerical
    - TEMP: numerical
    - PRES: numerical
    - Iws: numerical
    - Is: numerical
    - ...

    joins:
    - right: 'population'
      time_stamps: (population.date, population.date)
      relationship: 'many-to-many'
      memory: 86400.0
      lagged_targets: False

  population:
    columns:
    - DEWP: numerical
    - TEMP: numerical
    - PRES: numerical
    - Iws: numerical
    - Is: numerical
    - ...


container

  population
      subset   name          rows   type
  0   test     population    8661   View
  1   train    population   33096   View

  peripheral
      name          rows   type     
  0   population   41757   DataFrame

In [22]:
fast_prop = getml.feature_learning.FastProp(
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
    num_threads=1,
    aggregation=getml.feature_learning.FastProp.agg_sets.All,
)

predictor = getml.predictors.XGBoostRegressor(n_jobs=1)

pipe4 = getml.pipeline.Pipeline(
    tags=["memory: 1d", "simple features"],
    data_model=time_series4.data_model,
    feature_learners=[fast_prop],
    predictors=[predictor],
)

pipe4

Pipeline(data_model='population',
         feature_learners=['FastProp'],
         feature_selectors=[],
         include_categorical=False,
         loss_function=None,
         peripheral=['population'],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['memory: 1d', 'simple features'])

In [23]:
pipe4.check(time_series4.train)

Checking data model...

Staging...
[========================================] 100%


OK.


In [24]:
pipe4.fit(time_series4.train)

Checking data model...

Staging...
[========================================] 100%


OK.

Staging...
[========================================] 100%

FastProp: Trying 330 features...
[========================================] 100%

FastProp: Building features...
[========================================] 100%

XGBoost: Training as predictor...
[========================================] 100%


Trained pipeline.
Time taken: 0h:1m:1.504636



Pipeline(data_model='population',
         feature_learners=['FastProp'],
         feature_selectors=[],
         include_categorical=False,
         loss_function=None,
         peripheral=['population'],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['memory: 1d', 'simple features', 'container-531FXt'])

url: http://localhost:1709/#/getpipeline/air_pollution/pGebRB/0/

In [25]:
pipe4.score(time_series4.test)


Staging...
[========================================] 100%

FastProp: Building features...
[========================================] 100%




,date time,set used,target,mae,rmse,rsquared
0,2021-08-23 08:12:14,train,pm2.5,38.343,55.1887,0.6443
1,2021-08-23 08:12:19,test,pm2.5,44.1997,63.4947,0.545


### 3.5 Using featuretools

To make things a bit easier, we have written a high-level wrapper around featuretools which we placed in a separate module (`utils`).

In [26]:
data_train_pandas = time_series1.train.population.to_pandas()
data_test_pandas = time_series1.test.population.to_pandas()

tsfresh and featuretools require the time series to have ids. Since there is only a single time series, that series has the same id.

In [27]:
data_train_pandas["id"] = 1
data_test_pandas["id"] = 1

In [28]:
if RUN_FEATURETOOLS:
    ft_builder = FTTimeSeriesBuilder(
        num_features=200,
        horizon=pd.Timedelta(days=0),
        memory=pd.Timedelta(days=1),
        column_id="id",
        time_stamp="date",
        target="pm2.5",
    )
    #
    featuretools_training = ft_builder.fit(data_train_pandas)
    featuretools_test = ft_builder.transform(data_test_pandas)

    data_featuretools_training = getml.data.DataFrame.from_pandas(
        featuretools_training, name="featuretools_training"
    )
    data_featuretools_test = getml.data.DataFrame.from_pandas(
        featuretools_test, name="featuretools_test"
    )

In [29]:
if not RUN_FEATURETOOLS:
    data_featuretools_training = load_or_retrieve(
        "https://static.getml.com/datasets/air_pollution/featuretools/featuretools_training.csv"
    )
    data_featuretools_test = load_or_retrieve(
        "https://static.getml.com/datasets/air_pollution/featuretools/featuretools_test.csv"
    )

[========================================] 100%
[========================================] 100%


In [30]:
def set_roles_featuretools(df):
    df["date"] = df["date"].as_ts()
    df.set_role(["pm2.5"], getml.data.roles.target)
    df.set_role(["date"], getml.data.roles.time_stamp)
    df.set_role(df.roles.unused, getml.data.roles.numerical)
    df.set_role(["id"], getml.data.roles.unused_float)
    return df


df_featuretools_training = set_roles_featuretools(data_featuretools_training)
df_featuretools_test = set_roles_featuretools(data_featuretools_test)

In [31]:
predictor = getml.predictors.XGBoostRegressor()

pipe5 = getml.pipeline.Pipeline(
    tags=["featuretools", "memory: 1d"], predictors=[predictor]
)

pipe5

Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function=None,
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['featuretools', 'memory: 1d'])

In [32]:
pipe5.check(df_featuretools_training)

Checking data model...

Staging...
[========================================] 100%


OK.


In [33]:
pipe5.fit(df_featuretools_training)

Checking data model...

Staging...
[========================================] 100%


OK.

Staging...
[========================================] 100%

XGBoost: Training as predictor...
[========================================] 100%


Trained pipeline.
Time taken: 0h:0m:9.309428



Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function=None,
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['featuretools', 'memory: 1d'])

url: http://localhost:1709/#/getpipeline/air_pollution/ydiPTZ/0/

In [34]:
pipe5.score(df_featuretools_test)


Staging...
[========================================] 100%




,date time,set used,target,mae,rmse,rsquared
0,2021-08-23 08:12:38,featuretools_training,pm2.5,38.0455,54.4693,0.6567
1,2021-08-23 08:12:38,featuretools_test,pm2.5,45.3084,64.2717,0.5373


### 3.6 Using tsfresh

tsfresh is a rather low-level library. To make things a bit easier, we have written a high-level wrapper which we placed in a separate module (`utils`).

To limit the memory consumption, we undertake the following steps:

- We limit ourselves to a memory of 1 day from any point in time. This is necessary, because tsfresh duplicates records for every time stamp. That means that looking back 7 days instead of one day, the memory consumption would be  seven times as high.
- We extract only tsfresh's **MinimalFCParameters** and **IndexBasedFCParameters** (the latter is a superset of **TimeBasedFCParameters**).

In order to make sure that tsfresh's features can be compared to getML's features, we also do the following:

- We apply tsfresh's built-in feature selection algorithm.
- Of the remaining features, we only keep the 40 features most correlated with the target (in terms of the absolute value of the correlation).
- We add the original columns as additional features.


In [35]:
data_train_pandas

,date,pm2.5,DEWP,TEMP,PRES,Iws,Is,Ir,id
0,2010-01-02 00:00:00,129.0,-16.0,-4.0,1020.0,1.79,0.0,0.0,1
1,2010-01-02 01:00:00,148.0,-15.0,-4.0,1020.0,2.68,0.0,0.0,1
2,2010-01-02 02:00:00,159.0,-11.0,-5.0,1021.0,3.57,0.0,0.0,1
3,2010-01-02 03:00:00,181.0,-7.0,-5.0,1022.0,5.36,1.0,0.0,1
4,2010-01-02 04:00:00,138.0,-7.0,-5.0,1022.0,6.25,2.0,0.0,1
...,...,...,...,...,...,...,...,...,...
33091,2013-12-31 19:00:00,22.0,-19.0,7.0,1013.0,114.87,0.0,0.0,1
33092,2013-12-31 20:00:00,18.0,-21.0,7.0,1014.0,119.79,0.0,0.0,1
33093,2013-12-31 21:00:00,23.0,-21.0,7.0,1014.0,125.60,0.0,0.0,1
33094,2013-12-31 22:00:00,20.0,-21.0,6.0,1014.0,130.52,0.0,0.0,1


One of the issues about tsfresh is that is actually requires more memory than allowed by MyBinder. We therefore have to remove the parts that relate to this.

In [36]:
if RUN_TSFRESH:
    tsfresh_builder = TSFreshBuilder(
        num_features=200, memory=24, column_id="id", time_stamp="date", target="pm2.5"
    )
    #
    tsfresh_training = tsfresh_builder.fit(data_train_pandas)
    tsfresh_test = tsfresh_builder.transform(data_test_pandas)
    #
    data_tsfresh_training = getml.data.DataFrame.from_pandas(
        tsfresh_training, name="tsfresh_training"
    )
    data_tsfresh_test = getml.data.DataFrame.from_pandas(
        tsfresh_test, name="tsfresh_test"
    )

tsfresh does not contain built-in machine learning algorithms. In order to ensure a fair comparison, we use the exact same machine learning algorithm we have also used for getML: An XGBoost regressor with all hyperparameters set to their default value.

In order to do so, we load the tsfresh features into the getML engine.

In [37]:
if not RUN_TSFRESH:
    data_tsfresh_training = load_or_retrieve(
        "https://static.getml.com/datasets/air_pollution/tsfresh/tsfresh_training.csv"
    )
    data_tsfresh_test = load_or_retrieve(
        "https://static.getml.com/datasets/air_pollution/tsfresh/tsfresh_test.csv"
    )

[========================================] 100%
[========================================] 100%


As usual, we need to set roles:

In [38]:
def set_roles_tsfresh(df):
    df["date"] = df["date"].as_ts()
    df.set_role(["pm2.5"], getml.data.roles.target)
    df.set_role(["date"], getml.data.roles.time_stamp)
    df.set_role(df.roles.unused, getml.data.roles.numerical)
    df.set_role(["id"], getml.data.roles.unused_float)
    return df


df_tsfresh_training = set_roles_tsfresh(data_tsfresh_training)
df_tsfresh_test = set_roles_tsfresh(data_tsfresh_test)

In this case, our pipeline is very simple. It only consists of a single XGBoostRegressor.

In [39]:
predictor = getml.predictors.XGBoostRegressor()

pipe6 = getml.pipeline.Pipeline(tags=["tsfresh", "memory: 1d"], predictors=[predictor])

pipe6

Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function=None,
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['tsfresh', 'memory: 1d'])

In [40]:
pipe6.check(df_tsfresh_training)

Checking data model...

Staging...
[========================================] 100%


OK.


In [41]:
pipe6.fit(df_tsfresh_training)

Checking data model...

Staging...
[========================================] 100%


OK.

Staging...
[========================================] 100%

XGBoost: Training as predictor...
[========================================] 100%


Trained pipeline.
Time taken: 0h:0m:7.016484



Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function=None,
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['tsfresh', 'memory: 1d'])

url: http://localhost:1709/#/getpipeline/air_pollution/9ErjPw/0/

In [42]:
pipe6.score(df_tsfresh_test)


Staging...
[========================================] 100%




,date time,set used,target,mae,rmse,rsquared
0,2021-08-23 08:12:53,tsfresh_training,pm2.5,40.8063,57.7874,0.6106
1,2021-08-23 08:12:53,tsfresh_test,pm2.5,46.698,65.9163,0.5105


### 2.7 Productionization

It is possible to productionize the pipeline by transpiling the features into production-ready SQL code. Please also refer to getML's `sqlite3` module.

In [43]:
# Creates a folder named air_pollution_pipeline containing
# the SQL code.
pipe1.features.to_sql().save("air_pollution_pipeline")

## 4. Discussion

We have seen that getML outperforms tsfresh by more than 10 percentage points in terms of R-squared. We now want to analyze why that is.

There are two possible hypotheses:

- getML outperforms featuretools and tsfresh, because it using feature learning and is able to produce more complex features
- getML outperforms featuretools and tsfresh, because it makes better use of memory and is able to look back further.

Let's summarize our findings:


Name         | Memory  | Feature complexity | R-squared | RMSE |
------------ | ------- | ------------------ | --------- | ---- |
Pipeline 1   |  7 days |            complex |     62.3% | 57.8 |
Pipeline 2   |   1 day |            complex |     51.5% | 66.1 |
Pipeline 3   |  7 days |             simple |     52.0% | 65.7 |
Pipeline 4   |   1 day |             simple |     54.6% | 63.6 |
featuretools |   1 day |             simple |     53.7% | 64.3 |
tsfresh      |   1 day |             simple |     51.1% | 66.0 |


The summary table shows that combination of both of our hypotheses explains why getML outperforms featuretools and tsfresh. Complex features do better than simple features with a memory of one day. With a memory of seven days, simple features actually get worse. But when you look back seven days and allow more complex features, you get good results.

This suggests that getML outperforms featuretools and tsfresh, because it can make more efficient use of memory and thus look back further. Because RelMT uses feature learning and can build more complex features it can make better use of the greater look-back window.

## 5. Conclusion

We have compared getML's feature learning algorithms to tsfresh's brute-force feature engineering approaches on a data set related to air pollution in China. We found that getML significantly outperforms featuretools and tsfresh. These results are consistent with the view that feature learning can yield significant improvements over simple propositionalization approaches.

However, there are other datasets on which simple propositionalization performs well. Our suggestion is therefore to think of algorithms like `FastProp` and `RelMT` as tools in a toolbox. If a simple tool like `FastProp` gets the job done, then use that. But when you need more advanced approaches, like `RelMT`, you should have them at your disposal as well.

You are encouraged to reproduce these results. You will need getML (https://getml.com/product) and tsfresh (https://tsfresh.readthedocs.io/en/latest/). You can download both for free.

# Next Steps

This tutorial applied getML's feature learning algorithms RelMT and Relboost to a data set with many columns.

If you are interested in further real-world applications of getML, head back to the [notebook overview](welcome.md) and choose one of the remaining examples.

Here is some additional material from our [documentation](https://docs.getml.com/latest/) if you want to learn more about getML:
* [Feature learning with Multirel](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#multirel)
* [Feature learning with Relboost](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#relboost)


# Get in contact

If you have any question schedule a [call with Alex](https://go.getml.com/meetings/alexander-uhlig/getml-demo), the co-founder of getML, or write us an [email](team@getml.com). Prefer a private demo of getML? Just contact us to make an appointment.